# Scraping the web for corpus data
This notebook contains the code used to gather song lyrics for our corpus. 

Hamilton lyrics are sourced from http://www.themusicallyrics.com

In [149]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment, NavigableString
import re
import time

nonos = re.compile("[,.'#--()/]")

### Gathering Links for Hamilton Songs

In [ ]:
BASE_URL = "http://www.themusicallyrics.com"

conn = urlopen("http://www.themusicallyrics.com/h/351-hamilton-the-musical.html")
html = conn.read()
soup = BeautifulSoup(html, "html5lib")

# get links to lyrics for all songs

body = soup.find("tbody")
tbody = body.find("tbody") # lyrics are in nested tbody element
links = tbody.findAll("a")

# write links to a file
with open("hamilton-links", "w") as hl:
    for tag in links:
        link = tag.get("href")
        if link is not None:
            if link.startswith("/h/") and "synopsis" not in link:
                lyrics_page = BASE_URL + link
                hl.write(lyrics_page + "\n")

### Parsing each link and scraping lyrics

Note: the lyric files genereated below will still need to undergo some preprocessing before they're ready to be imported into the corpus

In [80]:
# open links on by one and strip lyrics
# I separated this step out to improve clarity
with open("hamilton-links", "r") as hlf:
    for link in hlf:
        conn = urlopen(link)
        soup = BeautifulSoup(conn.read(), "html5lib")
        title = soup.title.contents[0]
        filename = "lyrics/hamilton/raw/"
        filename += "".join(title.split()[2:-1])
        filename += ".txt"
        filename = re.sub(nonos, "", filename)
        p = soup.p
        with open(filename, "w") as hll:
            for item in p:
                if type(item) == NavigableString:   
                    hll.write(item + "\n")

NameError: name 'bs4' is not defined

### Gathering Links for Kendrick Lamar

In [91]:
BASE_URL = "https://www.azlyrics.com/"
CATALOG_EXT = "k/kendricklamar.html"

html = urlopen(BASE_URL + CATALOG_EXT).read()

soup = BeautifulSoup(html, "html5lib")

catalog = soup.find("div", id="listAlbum")
songs = catalog.findAll("a")

with open("song_lyric_links/kendrick_lamar-links.txt", "w") as kll:
    for song in songs:
        link = song.get("href")
        if link is not None:
            if link.startswith(".."):
                link = BASE_URL + link.lstrip("../") + "\n"
                kll.write(link)

### Scraping the lyrics

In [150]:
BASE_DIR = "lyrics/kendrick_lamar/"
with open("song_lyric_links/kendrick_lamar-links.txt", "r") as kll:
    for link in kll:
        html = urlopen(link).read()
        soup = BeautifulSoup(html, "html5lib")
        comments = soup.findAll(text=lambda text:isinstance(text, Comment))
        title = soup.title.contents[0].split()
        title = "".join(title[4:])
        title = re.sub(nonos, "", title)
        filename = BASE_DIR + title + ".txt"
        print(filename)
        for comment in comments:
            if "Usage of azlyrics.com" in comment: # lyrics follow this comment
                parent = comment.parent
                with open(filename, "w") as f:
                    for item in parent:
                        if type(item) == NavigableString:
                            if len(item) > 1: # ignore blank lines
                                f.write(item)
        time.sleep(3)

lyrics/kendrick_lamar/IsItLove.txt
lyrics/kendrick_lamar/Celebration.txt
lyrics/kendrick_lamar/PP.txt
lyrics/kendrick_lamar/SheNeedsMe.txt
lyrics/kendrick_lamar/IAmInterlude.txt
lyrics/kendrick_lamar/WannaBeHeard.txt
lyrics/kendrick_lamar/IDoThis.txt
lyrics/kendrick_lamar/UncleBobbyJasonKeaton.txt
lyrics/kendrick_lamar/Faith.txt
lyrics/kendrick_lamar/Trip.txt
lyrics/kendrick_lamar/VanitySlaves.txt
lyrics/kendrick_lamar/FarFromHere.txt
lyrics/kendrick_lamar/Thanksgiving.txt
lyrics/kendrick_lamar/LetMeBeMe.txt
lyrics/kendrick_lamar/Determined.txt
lyrics/kendrick_lamar/TheHeartPt2.txt
lyrics/kendrick_lamar/GrowingApartToGetCloser.txt
lyrics/kendrick_lamar/NightOfTheLivingJunkies.txt
lyrics/kendrick_lamar/PP15.txt
lyrics/kendrick_lamar/AlienGirlTodaywHer.txt
lyrics/kendrick_lamar/OppositesAttractTomorrowwoHer.txt
lyrics/kendrick_lamar/MichaelJordan.txt
lyrics/kendrick_lamar/IgnoranceIsBliss.txt
lyrics/kendrick_lamar/ROTCInterlude.txt
lyrics/kendrick_lamar/BarbedWire.txt
lyrics/kendrick_lam

RemoteDisconnected: Remote end closed connection without response

In [147]:
title = "AlienGirlTodayw/Her"
print(re.sub(nonos, "", title))

AlienGirlTodaywHer
